In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from models.darknet import Darknet

import os

In [2]:
PATH = "/home/paperspace/code/fontastic/data/stratified/"


In [3]:
train_path = os.path.join(PATH, 'train')
test_path = os.path.join(PATH, 'test')

In [4]:
train_path

'/home/paperspace/code/fontastic/data/stratified/train'

In [5]:
os.listdir(train_path)

['Fahkwang', 'MajorMonoDisplay', 'Lato', 'Merriweather', 'Lora', 'Lobster']

In [6]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.ImageFolder(root=train_path, 
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root=test_path, 
                                        transform=transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

In [9]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
print(trainloader.dataset.classes)
print(trainloader.dataset.class_to_idx)
print(' '.join('%5s' % trainloader.dataset.classes[labels[j]] for j in range(16)))

In [13]:
import torch.optim as optim

In [14]:
criterion = nn.CrossEntropyLoss()
m = Darknet([1, 2, 4, 6, 3], num_classes=len(trainloader.dataset.classes), nf=16)
m = nn.DataParallel(m, device_ids=None)
# m = nn.DataParallel(m, device_ids=None)


In [15]:
torch.device("cuda")

device(type='cuda')

In [16]:
device = torch.device("cuda")
net = m.to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-02)

In [17]:
1e-02 

0.01

In [18]:
1e-02 * 0.1 * 0.1

0.0001

In [42]:
def get_lr(i, lr=None, maxlr=1e-03):
    
    if i == 0:
        lr = maxlr * 0.1 * 0.1
    elif i <30:
        lr = lr / 0.1
    else:
        lr = lr * 0.1
    print(f"new lr {lr}")
    return lr

In [20]:
len(trainloader.dataset.imgs)

999

In [21]:
optimizer = optim.Adam(net.parameters(), lr=1e-02)

In [22]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    lr = None
#     lr = get_lr(i=0, lr=lr, maxlr=1e-04)
#     optimizer = optim.Adam(net.parameters(), lr=lr)
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        
#         lr = get_lr(i=i, lr=lr)
        optimizer.zero_grad()
        

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 200 mini-batches
            print('[epoch %d, mini-batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            
            #change lr
#             lr = get_lr(i=i, lr=lr, maxlr=1e-04)
#             optimizer = optim.Adam(net.parameters(), lr=lr)

            running_loss = 0.0

[epoch 1, mini-batch    10] loss: 2.185
[epoch 1, mini-batch    20] loss: 1.739
[epoch 1, mini-batch    30] loss: 1.520
[epoch 1, mini-batch    40] loss: 1.570
[epoch 1, mini-batch    50] loss: 1.425
[epoch 1, mini-batch    60] loss: 1.526
[epoch 2, mini-batch    10] loss: 1.381
[epoch 2, mini-batch    20] loss: 1.358
[epoch 2, mini-batch    30] loss: 1.375
[epoch 2, mini-batch    40] loss: 1.463
[epoch 2, mini-batch    50] loss: 1.268
[epoch 2, mini-batch    60] loss: 1.335
[epoch 3, mini-batch    10] loss: 1.245
[epoch 3, mini-batch    20] loss: 1.432
[epoch 3, mini-batch    30] loss: 1.121
[epoch 3, mini-batch    40] loss: 1.352
[epoch 3, mini-batch    50] loss: 1.280
[epoch 3, mini-batch    60] loss: 1.373
[epoch 4, mini-batch    10] loss: 1.227
[epoch 4, mini-batch    20] loss: 1.257
[epoch 4, mini-batch    30] loss: 1.170
[epoch 4, mini-batch    40] loss: 1.124
[epoch 4, mini-batch    50] loss: 1.296
[epoch 4, mini-batch    60] loss: 1.008
[epoch 5, mini-batch    10] loss: 1.119


In [23]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    lr = None
    lr = get_lr(i=0, lr=lr, maxlr=1e-04)
    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        
#         lr = get_lr(i=i, lr=lr)
        optimizer.zero_grad()
        

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 200 mini-batches
            print('[epoch %d, mini-batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            
            #change lr
            lr = get_lr(i=i, lr=lr, maxlr=1e-04)
            optimizer = optim.Adam(net.parameters(), lr=lr)

            running_loss = 0.0

new lr 1.0000000000000002e-07
[epoch 1, mini-batch    10] loss: 0.943
new lr 1.0000000000000002e-06
[epoch 1, mini-batch    20] loss: 1.058
new lr 1e-05
[epoch 1, mini-batch    30] loss: 0.995
new lr 0.0001
[epoch 1, mini-batch    40] loss: 1.028
new lr 1e-05
[epoch 1, mini-batch    50] loss: 0.983
new lr 1.0000000000000002e-06
[epoch 1, mini-batch    60] loss: 0.953
new lr 1.0000000000000002e-07
new lr 1.0000000000000002e-07
[epoch 2, mini-batch    10] loss: 1.024
new lr 1.0000000000000002e-06
[epoch 2, mini-batch    20] loss: 1.078
new lr 1e-05
[epoch 2, mini-batch    30] loss: 0.982
new lr 0.0001
[epoch 2, mini-batch    40] loss: 1.011
new lr 1e-05
[epoch 2, mini-batch    50] loss: 0.913
new lr 1.0000000000000002e-06
[epoch 2, mini-batch    60] loss: 0.930
new lr 1.0000000000000002e-07
new lr 1.0000000000000002e-07
[epoch 3, mini-batch    10] loss: 1.041
new lr 1.0000000000000002e-06
[epoch 3, mini-batch    20] loss: 0.929
new lr 1e-05
[epoch 3, mini-batch    30] loss: 1.059
new lr 

In [33]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    lr = None
    lr = get_lr(i=0, lr=lr, maxlr=1e-03)
    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        
#         lr = get_lr(i=i, lr=lr)
        optimizer.zero_grad()
        

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 200 mini-batches
            print('[epoch %d, mini-batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            
            #change lr
            lr = get_lr(i=i, lr=lr, maxlr=1e-03)
            optimizer = optim.Adam(net.parameters(), lr=lr)

            running_loss = 0.0

new lr 1.0000000000000002e-06
[epoch 1, mini-batch    10] loss: 1.046
new lr 1e-05
[epoch 1, mini-batch    20] loss: 0.938
new lr 0.0001
[epoch 1, mini-batch    30] loss: 0.866
new lr 0.001
[epoch 1, mini-batch    40] loss: 0.933
new lr 0.0001
[epoch 1, mini-batch    50] loss: 0.881
new lr 1e-05
[epoch 1, mini-batch    60] loss: 0.835
new lr 1.0000000000000002e-06
new lr 1.0000000000000002e-06
[epoch 2, mini-batch    10] loss: 0.828
new lr 1e-05
[epoch 2, mini-batch    20] loss: 0.831
new lr 0.0001
[epoch 2, mini-batch    30] loss: 0.853
new lr 0.001
[epoch 2, mini-batch    40] loss: 0.832
new lr 0.0001
[epoch 2, mini-batch    50] loss: 0.942
new lr 1e-05
[epoch 2, mini-batch    60] loss: 0.898
new lr 1.0000000000000002e-06
new lr 1.0000000000000002e-06
[epoch 3, mini-batch    10] loss: 0.815
new lr 1e-05
[epoch 3, mini-batch    20] loss: 0.847
new lr 0.0001
[epoch 3, mini-batch    30] loss: 0.895
new lr 0.001
[epoch 3, mini-batch    40] loss: 0.947
new lr 0.0001
[epoch 3, mini-batch  

In [45]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    lr = None
    lr = get_lr(i=0, lr=lr, maxlr=1e-02)
    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        
#         lr = get_lr(i=i, lr=lr)
        optimizer.zero_grad()
        

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 200 mini-batches
            print('[epoch %d, mini-batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            
            #change lr
            lr = get_lr(i=i, lr=lr, maxlr=1e-02)
            optimizer = optim.Adam(net.parameters(), lr=lr)

            running_loss = 0.0

new lr 0.0001
[epoch 1, mini-batch    20] loss: 0.898
new lr 0.001
[epoch 1, mini-batch    40] loss: 0.957
new lr 0.0001
[epoch 1, mini-batch    60] loss: 0.967
new lr 1e-05
new lr 0.0001
[epoch 2, mini-batch    20] loss: 0.917
new lr 0.001
[epoch 2, mini-batch    40] loss: 0.904
new lr 0.0001
[epoch 2, mini-batch    60] loss: 0.846
new lr 1e-05
new lr 0.0001
[epoch 3, mini-batch    20] loss: 0.853
new lr 0.001
[epoch 3, mini-batch    40] loss: 0.998
new lr 0.0001
[epoch 3, mini-batch    60] loss: 0.881
new lr 1e-05
new lr 0.0001
[epoch 4, mini-batch    20] loss: 0.839
new lr 0.001
[epoch 4, mini-batch    40] loss: 0.952
new lr 0.0001
[epoch 4, mini-batch    60] loss: 0.916
new lr 1e-05
new lr 0.0001
[epoch 5, mini-batch    20] loss: 0.812
new lr 0.001
[epoch 5, mini-batch    40] loss: 0.899
new lr 0.0001
[epoch 5, mini-batch    60] loss: 0.960
new lr 1e-05
new lr 0.0001
[epoch 6, mini-batch    20] loss: 0.833
new lr 0.001
[epoch 6, mini-batch    40] loss: 0.891
new lr 0.0001
[epoch 6,

In [51]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# # print images
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(16)))

In [52]:
outputs = net(images)


In [53]:
predicted

tensor([1, 1, 1, 1, 0, 0, 1], device='cuda:0')

In [54]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % trainloader.dataset.classes[labels[j]] for j in range(16)))
print('Predicted: ', ' '.join('%5s' % trainloader.dataset.classes[predicted[j]] for j in range(16)))

Predicted:   Lora Fahkwang  Lora  Lora Fahkwang  Lora  Lora  Lato Fahkwang Fahkwang Lobster  Lato Fahkwang  Lato  Lora  Lato
Predicted:  Merriweather Fahkwang Fahkwang  Lato Fahkwang Fahkwang  Lato  Lato Fahkwang Fahkwang Lobster  Lato Fahkwang  Lato  Lato  Lato


In [70]:
correct = 0
total = 0
all_lbls = []
all_preds = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_lbls.extend(labels.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 250 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 250 test images: 66 %


In [ ]:
all_preds

In [ ]:
for x,y in zip(all_lbls, all_preds):
    print(x,y)

In [73]:
from sklearn.metrics import confusion_matrix

In [74]:
confusion_matrix(all_lbls, all_preds)

array([[337,  42,   1,   4,   0,   0],
       [ 44, 231,   8,   5,  13,  19],
       [  2,  15,  14,   0,   0,   1],
       [ 48,  70,   5,   2,   0,   3],
       [  1,  13,   0,   0,  18,   0],
       [  1,  43,   0,   1,   0,  58]])